In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

In [15]:
# loading data
life_data_path = "Life Expectancy Data.csv"
life_df = pd.read_csv(life_data_path)
life_df.head(10)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
5,Afghanistan,2010,Developing,58.8,279.0,74,0.01,79.679367,66.0,1989,...,66.0,9.20,66.0,0.1,553.328940,2883167.0,18.4,18.4,0.448,9.2
6,Afghanistan,2009,Developing,58.6,281.0,77,0.01,56.762217,63.0,2861,...,63.0,9.42,63.0,0.1,445.893298,284331.0,18.6,18.7,0.434,8.9
7,Afghanistan,2008,Developing,58.1,287.0,80,0.03,25.873925,64.0,1599,...,64.0,8.33,64.0,0.1,373.361116,2729431.0,18.8,18.9,0.433,8.7
8,Afghanistan,2007,Developing,57.5,295.0,82,0.02,10.910156,63.0,1141,...,63.0,6.73,63.0,0.1,369.835796,26616792.0,19.0,19.1,0.415,8.4
9,Afghanistan,2006,Developing,57.3,295.0,84,0.03,17.171518,64.0,1990,...,58.0,7.43,58.0,0.1,272.563770,2589345.0,19.2,19.3,0.405,8.1


In [4]:
# data cleaning, this might be too aggressive. (removed U.S.A.)
# Maybe we extract only relevant columns, then do a dropna to hopefully preserve some data. 
clean_ld = life_df.dropna() 

In [5]:
# Summary of potential Life expctancy factors by nation
le_factors = ['Status','infant deaths', 'Life expectancy ', 
                        'Alcohol', 'percentage expenditure', 'Hepatitis B','Measles ', 
                        ' BMI ','Polio', 'Total expenditure','Diphtheria ', 
                        ' HIV/AIDS', 'GDP', 'Population',' thinness  1-19 years', 
                        ' thinness 5-9 years','Schooling']

inital_summary = clean_ld.groupby("Country")[le_factors].agg("max")
nation_statuses = clean_ld[["Country","Status"]].drop_duplicates()
summary_by_nation = pd.merge(inital_summary, nation_statuses , on=["Country"])

In [21]:
# Charting LE Result function
def chart_life_expectancy_scatter(factor):
    summary_by_nation.plot(x=factor,y='Life expectancy ',kind="scatter")
    
def chart_life_expectancy_lr(factor):
    x_values = summary_by_nation[factor]
    y_values = summary_by_nation["Life expectancy "]
    (slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    # # Label plot and annotate the line equation
    plt.xlabel(factor)
    plt.ylabel("Life expectancy ")
    print(f"The r-value is: {rvalue**2}")
    plt.show()
    
def life_expectancy_correlation(factor):
    x = summary_by_nation[factor]
    y = summary_by_nation["Life expectancy "]
    corr, p = st.stats.pearsonr(x, y)
    print(f"The correlation between {factor} and the life expectancy is {corr}.")
    return (corr)

def lifeexpectancy75():
    below75 = summary_by_nation.loc[(summary_by_nation["Life expectancy "]=<75),["Total expenditure"]]
    over75 = summary_by_nation.loc[(summary_by_nation["Life expectancy "]>75),["Total expenditure"]]
    mean_expenditure_below75 = below75["Total expenditure"].mean()
    mean_expenditure_over75 = over75["Total expenditure"].mean()
    d = {'Below 75': below75, 'Over 75': over75}
    data75 = pd.DataFrame(data=d)
    
    # Create a boxplot to compare means
    
    
    t_test = st.ttest_ind(below75, over75, equal_var=False)
    return(t_test)  


In [22]:

#chart_life_expectancy_scatter("GDP")
lifeexpectancy75()
#chart_life_expectancy_lr(le_factors[13])
# summary_by_nation["Population"]
# short_summ = summary_by_nation[["Country", "Population"]]
# short_summ.sort_values(inplace=True, by=["Population"])
# india = short_summ[short_summ["Country"] == "India"]
# indias data is wacky bonko nonsense sooooo we arent totally sure how reliable any of it is 

Ttest_indResult(statistic=array([-0.46142356]), pvalue=array([0.64811665]))

# We Were having a geat time. Made our nation based summary of potential le factors. We went
# to plot population against le and found a strnage looking graph, we then looked deeper
# and chris helped us see that our data is worth less than a car
# with no wheeels no engine no steering wheel and no gas
# so we might find another dataset. 
# reginald votes to ignore this reveletion as long as feasible and maybe just remove all the 
# dirty data.
